In [52]:
# load module and pre generate data
from graph_tree_module import Node, sim, graph_tree

import numpy as np

import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)


In [53]:
sliders_data = [{"id": gt.attr_name[i], "min": gt.X_maxmin[1][i], "max": gt.X_maxmin[0][i], "value": gt.root.center[i], "type": "f"} for i in range(len(gt.attr_name))]
# print(sliders_data)

recom_data = [{"i": i, "id": gt.attr_name[i], "max_val": gt.recom[i][0].center[i], "min_val": gt.recom[i][1].center[i]} for i in range(len(gt.attr_name))]
# print(recom_data)

status = gt.get_suround_cluster(gt.root, gt.root.center)
print(status)
def get_suround_node_data(status):
    cur_node = status["cur_cluster"]
    cur_cluster = [{"id": cur_node.id, "position": "center", "x": 300, "y": 200, "color": "red"}]

    contra_node = status["contra_cluster"]
    contra_cluster = [{"id": contra_node[i].id, "position": "left", "x": 100, "y": 150+i*100, "color": "green"} for i in range(len(contra_node))]

    super_node = status["super_cluster"]
    super_cluster = [{"id": super_node.id, "position": "top", "x": 300, "y": 100, "color": "dark red"}] if super_node else []

    sim_node = status["neibour_cluster"]
    sim_cluster = [{"id": sim_node[i].id, "position": "right", "x": 500, "y": 200+i*100, "color": "blue"} for i in range(len(sim_node))] if sim_node else []

    sub_node = status["sub_cluster"]
    sub_cluster = [{"id": sub_node[i].id, "position": "bottom", "x": 200+i*100, "y": 350, "color": "pink"} for i in range(len(sub_node))] if sub_node else []

    return cur_cluster + contra_cluster + super_cluster + sim_cluster + sub_cluster

graph_data = get_suround_node_data(status)

print(graph_data)

def get_tree_data(status):
    cur_node = status["cur_cluster"]
    up1_node = cur_node.up_lv if cur_node.up_lv else []
    up2_node = up1_node.up_lv if up1_node.up_lv else []

tree_data = {
    'name': 'Root',
    'children': [
        {'name': 'Child 1', 'children': [
            {'name': 'Grandchild 1'},
            {'name': 'Grandchild 2'}
        ]},
        {'name': 'Child 2', 'children': [
            {'name': 'Grandchild 3'}
        ]}
    ]
}

[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.250018

In [54]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders():
    return jsonify(sliders_data)

@app.route('/handle_slider', methods=['POST'])
def submit_slider_values():
    data = request.json
    print('Received slider values:', data)
    query = np.array([float(val) for val in data.values()])
    cur = gt.search(query)
    print(cur.id)
    print(gt.status)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Process the data as needed
    return jsonify({'status': 'slider respon success', 'received': data})
# slider end ################################################################

# recom ####################################################################
@app.route('/recom')
def recom():
    return jsonify(recom_data)

@app.route('/handle_recom', methods=['POST'])
def handle_recom():
    data = request.json
    button_id = int(data.get('buttonId'))
    val = int(data.get('val'))
    print(button_id, val)
    moved_node = gt.recom[button_id][val]
    gt.get_suround_cluster(moved_node)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Handle the button click based on the button_id
    return jsonify({'status': 'recom respon success', 'buttonId': button_id, 'val': val})
# recom end ####################################################################

@app.route('/graph-data')
def graph():
    return jsonify(graph_data)

@app.route('/tree-data')
def tree():
    return jsonify(tree_data)

if __name__ == '__main__':
    app.run(port=8123)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8123
Press CTRL+C to quit
127.0.0.1 - - [05/Sep/2024 15:41:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 15:41:27] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Sep/2024 15:41:27] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Sep/2024 15:41:27] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 15:41:27] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 15:41:27] "GET /graph-data HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 15:41:27] "GET /tree-data HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 15:42:15] "GET /graph-data HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 15:42:15] "POST /handle_slider HTTP/1.1" 200 -


Received slider values: {'Popularityslider': '0', 'danceabilityslider': '0.5299', 'energyslider': '0.07301', 'keyslider': '7.05', 'loudnessslider': '0.756', 'modeslider': '0.18', 'speechinessslider': '0.0625', 'acousticnessslider': '0.69', 'instrumentalnessslider': '0.59', 'livenessslider': '0.6536', 'valenceslider': '0.4759', 'temposlider': '34.132', 'duration_in min/msslider': '0.50165', 'time_signatureslider': '3.93'}
checking 0, dist: 3.9254550547803193
0 potential
[1, 2] 

checking 1, dist: 4.292665539435567
1 potential
[2, 3, 4] 

checking 2, dist: 3.635461980833771
cur node: 2, closest: 3.635461980833771
[3, 4, 5, 6] 

checking 3, dist: 4.622379636211933
3 potential
[4, 5, 6, 7, 8, 9] 

checking 4, dist: 3.6378579995691394
4 potential
[5, 6, 7, 8, 9, 10, 11] 

checking 5, dist: 3.953433455924567
5 potential
[6, 7, 8, 9, 10, 11, 14, 12, 13] 

checking 6, dist: 2.9424111133301647
cur node: 6, closest: 2.9424111133301647
[7, 8, 9, 10, 11, 14, 12, 13, 16, 15] 

checking 7, dist: 4.5

127.0.0.1 - - [05/Sep/2024 15:42:38] "GET /graph-data HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 15:42:38] "POST /handle_slider HTTP/1.1" 200 -


Received slider values: {'Popularityslider': '0', 'danceabilityslider': '0.5299', 'energyslider': '0.07301', 'keyslider': '7.05', 'loudnessslider': '0.756', 'modeslider': '0.18', 'speechinessslider': '0.0625', 'acousticnessslider': '0.69', 'instrumentalnessslider': '0.59', 'livenessslider': '0.6536', 'valenceslider': '0.4759', 'temposlider': '181.632', 'duration_in min/msslider': '1044963.88165', 'time_signatureslider': '3.93'}
checking 0, dist: 4.22353166544238
0 potential
[1, 2] 

checking 1, dist: 4.598644093469034
1 potential
[2, 3, 4] 

checking 2, dist: 3.919551495057248
cur node: 2, closest: 3.919551495057248
[3, 4, 5, 6] 

checking 3, dist: 4.867832753475459
3 potential
[4, 5, 6, 7, 8, 9] 

checking 4, dist: 4.114349074395348
4 potential
[5, 6, 7, 8, 9, 10, 11] 

checking 5, dist: 4.195881700434146
5 potential
[6, 7, 8, 9, 10, 11, 14, 12, 13] 

checking 6, dist: 3.360472656516344
cur node: 6, closest: 3.360472656516344
[7, 8, 9, 10, 11, 14, 12, 13, 16, 15] 

checking 7, dist: 4

127.0.0.1 - - [05/Sep/2024 15:43:07] "POST /handle_recom HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 15:43:07] "GET /graph-data HTTP/1.1" 200 -


0 1
[0.27265133 0.65713275 0.69860278 0.68421053 0.7815268  1.
 0.07244355 0.10668186 0.01853785 0.14165524 0.75270519 0.46148395
 0.13974785 0.7406015 ] [0.49171717 0.62277607 0.72619186 0.70396694 0.77905357 1.
 0.06700413 0.14214463 0.02870229 0.19458671 0.71778087 0.4763506
 0.13137124 0.74272727]
[0.27265133 0.65713275 0.69860278 0.68421053 0.7815268  1.
 0.07244355 0.10668186 0.01853785 0.14165524 0.75270519 0.46148395
 0.13974785 0.7406015 ] [0.47134111 0.52212517 0.74900149 0.7087966  0.78768744 1.
 0.06280401 0.10119061 0.02865018 0.20309752 0.52117946 0.50408515
 0.14839712 0.73696043]
[0.27265133 0.65713275 0.69860278 0.68421053 0.7815268  1.
 0.07244355 0.10668186 0.01853785 0.14165524 0.75270519 0.46148395
 0.13974785 0.7406015 ] [0.49171717 0.62277607 0.72619186 0.70396694 0.77905357 1.
 0.06700413 0.14214463 0.02870229 0.19458671 0.71778087 0.4763506
 0.13137124 0.74272727]
[0.27265133 0.65713275 0.69860278 0.68421053 0.7815268  1.
 0.07244355 0.10668186 0.01853785 0.141

127.0.0.1 - - [05/Sep/2024 15:43:17] "POST /handle_recom HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 15:43:17] "GET /graph-data HTTP/1.1" 200 -


6 1
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.12227291 0.27099151 0.4585729
 0.09641026 0.67329545] [0.4101459  0.49256889 0.36862392 0.16727273 0.68366886 1.
 0.03219391 0.6608523  0.02663339 0.14847234 0.29189958 0.45699643
 0.11275586 0.68333333]
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.12227291 0.27099151 0.4585729
 0.09641026 0.67329545] [0.38355465 0.53743531 0.40269881 0.19203269 0.68464131 1.
 0.0448142  0.68572887 0.02580899 0.1602539  0.43829216 0.44910447
 0.10385525 0.6994382 ]
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.12227291 0.27099151 0.4585729
 0.09641026 0.67329545] [0.4101459  0.49256889 0.36862392 0.16727273 0.68366886 1.
 0.03219391 0.6608523  0.02663339 0.14847234 0.29189958 0.45699643
 0.11275586 0.68333333]
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.1222